In [1]:
import os
import torch
from tqdm import tqdm
import pandas as pd

In [2]:
DATA_ROOT = '../../dataset'
TEST_DATASET_PATH = os.path.join(DATA_ROOT, 'test')

INFERENCE_SIZE = 1280
AUGMENT = True

IMAGES = []
for image_name in os.listdir(TEST_DATASET_PATH):
    if not image_name.startswith('._'):
        IMAGES.append(os.path.join(TEST_DATASET_PATH, image_name))

IMAGES.sort()

In [15]:
project = 'bc_lv2-object_detection'
name = 'yolov5l6_pretrained_highaug_imgsize1280_pseudofinal_fold2_HONEY'

In [16]:
model = torch.hub.load('./', 'custom', path = f'{project}/{name}/weights/best.pt', source = 'local')
model.conf = 0.001  # NMS confidence threshold
model.iou = 0.6  # NMS IoU threshold
# model.agnostic = False  # NMS class-agnostic
# model.multi_label = False  # NMS multiple labels per box
# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
# model.max_det = 1000  # maximum number of detections per image
# model.amp = False  # Automatic Mixed Precision (AMP) inference

YOLOv5 🚀 2022-11-30 Python-3.7.15 torch-1.7.1 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)

Fusing layers... 
YOLOv5l6-custom summary: 346 layers, 76187892 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 


In [17]:
prediction_strings = []
file_names = []

for i in tqdm(range(len(IMAGES))):
    prediction_string = ''
    
    output = model(IMAGES[i], size=INFERENCE_SIZE, augment=AUGMENT)
    for bbox in output.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, conf, cls, _ = bbox
        prediction_string += f'{cls} {conf} {xmin} {ymin} {xmax} {ymax} '
    
    path_split = IMAGES[i].split('/')
    image_path = f'{path_split[-2]}/{path_split[-1]}'

    prediction_strings.append(prediction_string)
    file_names.append(image_path)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

100%|██████████| 4871/4871 [12:08<00:00,  6.68it/s]


,PredictionString,image_id
0,7 0.9888781309127808 601.2654418945312 516.880...,test/0000.jpg
1,5 0.9822874069213867 343.31988525390625 249.23...,test/0001.jpg
2,1 0.9731317758560181 75.2083969116211 272.3052...,test/0002.jpg
3,9 0.9691352844238281 144.3590087890625 263.903...,test/0003.jpg
4,1 0.9874654412269592 190.96458435058594 252.47...,test/0004.jpg


In [18]:
submission

,PredictionString,image_id
0,7 0.9888781309127808 601.2654418945312 516.880...,test/0000.jpg
1,5 0.9822874069213867 343.31988525390625 249.23...,test/0001.jpg
2,1 0.9731317758560181 75.2083969116211 272.3052...,test/0002.jpg
3,9 0.9691352844238281 144.3590087890625 263.903...,test/0003.jpg
4,1 0.9874654412269592 190.96458435058594 252.47...,test/0004.jpg
...,...,...
4866,0 0.9465426206588745 295.0429382324219 345.914...,test/4866.jpg
4867,3 0.9819993376731873 416.7394104003906 315.297...,test/4867.jpg
4868,7 0.9690581560134888 114.2170181274414 450.831...,test/4868.jpg
4869,0 0.9812177419662476 0.0 239.2442626953125 102...,test/4869.jpg


In [19]:
model = 'yolov5l6'
fold = 'f2'
conf_thres = '1e-3'

In [20]:
submission.to_csv(f'results/submission/result_{model}_{fold}_pseudo_{conf_thres}.csv', index = None)